In [1]:
import pandas as pd
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 200)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix
from sklearn.utils import resample
from xgboost import XGBClassifier

import pickle


In [2]:
df = pd.read_csv('../data/california_wildfires.csv')

In [3]:
# Create dummy variables for the county column
counties = pd.get_dummies(df.county, drop_first = True)
# Drop county column along with unnecessary columns (Unnamed columns, year, and acres burned)
df2 = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'county', 'year', 'acres_burned'], axis = 1)

In [4]:
# Feature engineer month column from the date column
df2['month'] = pd.DatetimeIndex(df2['date']).month
# Drop the date column
df2.drop(columns = ['date'], axis = 1, inplace = True)

In [5]:
# Create dummy variables for the months
month = pd.get_dummies(df2.month, prefix = 'month_', drop_first = True)
# Drop the month column
df2.drop(columns = 'month', axis =1, inplace = True)

In [6]:
# Combine the original dataframe with the dummy variables
df2 = pd.concat([df2, counties, month], axis = 1)

In [7]:
# Split dataset into target variable and features
y = df2.fire_started
X = df2.drop(columns = ['fire_started'], axis = 1)

In [8]:
list(X.columns)

['Alfalfa & Hay_acres',
 'Alfalfa & Hay_percentage',
 'Almonds_acres',
 'Almonds_percentage',
 'Barren_acres',
 'Barren_percentage',
 'Corn_acres',
 'Corn_percentage',
 'Cotton_acres',
 'Cotton_percentage',
 'Deciduous Forest_acres',
 'Deciduous Forest_percentage',
 'Evergreen Forest_acres',
 'Evergreen Forest_percentage',
 'Fallow_acres',
 'Fallow_percentage',
 'Fruit Trees_acres',
 'Fruit Trees_percentage',
 'Grain Crops_acres',
 'Grain Crops_percentage',
 'Grapes_acres',
 'Grapes_percentage',
 'Grassland_acres',
 'Grassland_percentage',
 'High Intensity Developed_acres',
 'High Intensity Developed_percentage',
 'Low Intensity Developed_acres',
 'Low Intensity Developed_percentage',
 'Mixed Forest_acres',
 'Mixed Forest_percentage',
 'Other Ocean/Mexico_acres',
 'Other Ocean/Mexico_percentage',
 'Other Tree Crops_acres',
 'Other Tree Crops_percentage',
 'Other_acres',
 'Other_percentage',
 'Rice_acres',
 'Rice_percentage',
 'Shrubland_acres',
 'Shrubland_percentage',
 'Tomatoes_acres

In [9]:
df_features = X[list(X.columns)]

In [10]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(df_features)
poly2_columns = poly_2.get_feature_names(df_features.columns)
df_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)
df_poly2.head()

,Alfalfa & Hay_acres,Alfalfa & Hay_percentage,Almonds_acres,Almonds_percentage,Barren_acres,Barren_percentage,Corn_acres,Corn_percentage,Cotton_acres,Cotton_percentage,Deciduous Forest_acres,Deciduous Forest_percentage,Evergreen Forest_acres,Evergreen Forest_percentage,Fallow_acres,Fallow_percentage,Fruit Trees_acres,Fruit Trees_percentage,Grain Crops_acres,Grain Crops_percentage,Grapes_acres,Grapes_percentage,Grassland_acres,Grassland_percentage,High Intensity Developed_acres,High Intensity Developed_percentage,Low Intensity Developed_acres,Low Intensity Developed_percentage,Mixed Forest_acres,Mixed Forest_percentage,Other Ocean/Mexico_acres,Other Ocean/Mexico_percentage,Other Tree Crops_acres,Other Tree Crops_percentage,Other_acres,Other_percentage,Rice_acres,Rice_percentage,Shrubland_acres,Shrubland_percentage,Tomatoes_acres,Tomatoes_percentage,Vegs & Fruits_acres,Vegs & Fruits_percentage,Walnuts_acres,Walnuts_percentage,Water_acres,Water_percentage,Wetlands_acres,Wetlands_percentage,Winter Wheat_acres,Winter Wheat_percentage,max_elevation,min_elevation,Avg Air Temp (F)_Weekly,Avg Rel Hum (%)_Weekly,Avg Wind Speed (mph)_Weekly,Dew Point (F)_Weekly,Max Air Temp (F)_Weekly,Max Rel Hum (%)_Weekly,Min Air Temp (F)_Weekly,Min Rel Hum (%)_Weekly,Precip (in)_Weekly,Avg Air Temp (F)_month,Avg Rel Hum (%)_month,Avg Wind Speed (mph)_month,Dew Point (F)_month,Max Air Temp (F)_month,Max Rel Hum (%)_month,Min Air Temp (F)_month,Min Rel Hum (%)_month,Precip (in)_month,Population,county_acres,pop_density,Alpine,Amador,Butte,Calaveras,Colusa,Contra Costa,Del Norte,El Dorado,Fresno,Glenn,Humboldt,Imperial,Inyo,Kern,Kings,Lake,Lassen,Los Angeles,Madera,Marin,Mariposa,Mendocino,Merced,Modoc,Mono,Monterey,Napa,Nevada,Orange,Placer,Plumas,Riverside,Sacramento,San Benito,San Bernardino,San Diego,San Francisco,San Joaquin,San Luis Obispo,San Mateo,Santa Barbara,Santa Clara,Santa Cruz,Shasta,Sierra,Siskiyou,Solano,Sonoma,Stanislaus,Sutter,Tehama,Trinity,Tulare,Tuolumne,Ventura,Yolo,Yuba,month__2,month__3,month__4,month__5,month__6,month__7,month__8,month__9,month__10,month__11,month__12,Alfalfa & Hay_acres^2,Alfalfa & Hay_acres Alfalfa & Hay_percentage,Alfalfa & Hay_acres Almonds_acres,Alfalfa & Hay_acres Almonds_percentage,Alfalfa & Hay_acres Barren_acres,Alfalfa & Hay_acres Barren_percentage,Alfalfa & Hay_acres Corn_acres,...,Trinity Ventura,Trinity Yolo,Trinity Yuba,Trinity month__2,Trinity month__3,Trinity month__4,Trinity month__5,Trinity month__6,Trinity month__7,Trinity month__8,Trinity month__9,Trinity month__10,Trinity month__11,Trinity month__12,Tulare^2,Tulare Tuolumne,Tulare Ventura,Tulare Yolo,Tulare Yuba,Tulare month__2,Tulare month__3,Tulare month__4,Tulare month__5,Tulare month__6,Tulare month__7,Tulare month__8,Tulare month__9,Tulare month__10,Tulare month__11,Tulare month__12,Tuolumne^2,Tuolumne Ventura,Tuolumne Yolo,Tuolumne Yuba,Tuolumne month__2,Tuolumne month__3,Tuolumne month__4,Tuolumne month__5,Tuolumne month__6,Tuolumne month__7,Tuolumne month__8,Tuolumne month__9,Tuolumne month__10,Tuolumne month__11,Tuolumne month__12,Ventura^2,Ventura Yolo,Ventura Yuba,Ventura month__2,Ventura month__3,Ventura month__4,Ventura month__5,Ventura month__6,Ventura month__7,Ventura month__8,Ventura month__9,Ventura month__10,Ventura month__11,Ventura month__12,Yolo^2,Yolo Yuba,Yolo month__2,Yolo month__3,Yolo month__4,Yolo month__5,Yolo month__6,Yolo month__7,Yolo month__8,Yolo month__9,Yolo month__10,Yolo month__11,Yolo month__12,Yuba^2,Yuba month__2,Yuba month__3,Yuba month__4,Yuba month__5,Yuba month__6,Yuba month__7,Yuba month__8,Yuba month__9,Yuba month__10,Yuba month__11,Yuba month__12,month__2^2,month__2 month__3,month__2 month__4,month__2 month__5,month__2 month__6,month__2 month__7,month__2 month__8,month__2 month__9,month__2 month__10,month__2 month__11,month__2 month__12,month__3^2,month__3 month__4,month__3 month__5,month__3 month__6,month__3 month__7,month__3 month__8,month__3 month__9,month__3 month__10,month__3 month__11,mont

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_poly2, y, random_state = 0)

In [12]:
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

selector = SelectKBest(f_classif, k=200)

selector.fit(X_train, y_train)

/Users/malcolmkatzenbach/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [  501   529   537 ... 10434 10435 10437] are constant.
  UserWarning)
/Users/malcolmkatzenbach/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


SelectKBest(k=200)

In [13]:
selected_columns = X_train.columns[selector.get_support()]
removed_columns = X_train.columns[~selector.get_support()]

In [14]:
list(selected_columns)

['Low Intensity Developed_acres',
 'Avg Air Temp (F)_Weekly',
 'Avg Rel Hum (%)_Weekly',
 'Dew Point (F)_Weekly',
 'Max Air Temp (F)_Weekly',
 'Min Air Temp (F)_Weekly',
 'Avg Air Temp (F)_month',
 'Avg Rel Hum (%)_month',
 'Dew Point (F)_month',
 'Max Air Temp (F)_month',
 'Min Air Temp (F)_month',
 'Min Rel Hum (%)_month',
 'Precip (in)_month',
 'county_acres',
 'month__7',
 'Barren_acres Grassland_acres',
 'Barren_acres month__7',
 'Barren_percentage month__7',
 'Deciduous Forest_acres month__7',
 'Deciduous Forest_acres month__8',
 'Evergreen Forest_acres Population',
 'Evergreen Forest_acres month__7',
 'Evergreen Forest_acres month__8',
 'Evergreen Forest_percentage month__7',
 'Fallow_acres month__7',
 'Grassland_acres Low Intensity Developed_acres',
 'Grassland_acres Shrubland_acres',
 'Grassland_acres Shrubland_percentage',
 'Grassland_acres max_elevation',
 'Grassland_acres Avg Air Temp (F)_Weekly',
 'Grassland_acres Avg Wind Speed (mph)_Weekly',
 'Grassland_acres Dew Point (

In [106]:
ch_feat=['Deciduous Forest_acres month__7',
         'Deciduous Forest_acres month__8',
         'Evergreen Forest_acres Population',
         'Evergreen Forest_acres month__7',
         'Evergreen Forest_acres month__8',
         'Evergreen Forest_percentage month__7',
         'Fallow_acres month__7',
         'Grassland_acres Low Intensity Developed_acres',
         'Grassland_acres Shrubland_acres',
         'Grassland_acres Shrubland_percentage',
         'Grassland_acres max_elevation',
         'Grassland_acres Avg Air Temp (F)_Weekly',
         'Grassland_acres Avg Wind Speed (mph)_Weekly',
         'Grassland_acres Dew Point (F)_Weekly',
         'Grassland_acres Max Air Temp (F)_Weekly',
         'Grassland_acres Min Air Temp (F)_Weekly',
         'Grassland_acres Avg Air Temp (F)_month',
         'Grassland_acres Avg Wind Speed (mph)_month',
         'Grassland_acres Dew Point (F)_month',
         'Grassland_acres Max Air Temp (F)_month',
         'Grassland_acres Min Air Temp (F)_month',
         'Grassland_acres month__6',
         'Grassland_acres month__7',
         'Grassland_percentage Shrubland_acres',
         'Grassland_percentage month__7',
         'Low Intensity Developed_acres Shrubland_percentage',
         'Low Intensity Developed_acres max_elevation',
         'Low Intensity Developed_acres Avg Air Temp (F)_Weekly',
         'Low Intensity Developed_acres Avg Wind Speed (mph)_Weekly',
         'Low Intensity Developed_acres Dew Point (F)_Weekly',
         'Low Intensity Developed_acres Max Air Temp (F)_Weekly',
         'Low Intensity Developed_acres Max Rel Hum (%)_Weekly',
         'Low Intensity Developed_acres Min Air Temp (F)_Weekly',
         'Low Intensity Developed_acres Avg Air Temp (F)_month',
         'Low Intensity Developed_acres Avg Wind Speed (mph)_month',
         'Low Intensity Developed_acres Dew Point (F)_month',
         'Low Intensity Developed_acres Max Air Temp (F)_month',
         'Low Intensity Developed_acres Max Rel Hum (%)_month',
         'Low Intensity Developed_acres Min Air Temp (F)_month',
         'Low Intensity Developed_acres county_acres',
         'Low Intensity Developed_acres month__6',
         'Low Intensity Developed_acres month__7',
         'Low Intensity Developed_acres month__8',
         'Low Intensity Developed_percentage Shrubland_acres',
         'Low Intensity Developed_percentage county_acres',
         'Mixed Forest_acres month__7',
         'Shrubland_acres Avg Air Temp (F)_Weekly',
         'Shrubland_acres Dew Point (F)_Weekly',
         'Shrubland_acres Min Air Temp (F)_Weekly',
         'Shrubland_acres Avg Air Temp (F)_month',
         'Shrubland_acres Dew Point (F)_month',
         'Shrubland_acres Min Air Temp (F)_month',
         'Shrubland_acres month__7',
         'Shrubland_percentage Avg Air Temp (F)_Weekly',
         'Shrubland_percentage Dew Point (F)_Weekly',
         'Shrubland_percentage Max Air Temp (F)_Weekly',
         'Shrubland_percentage Min Air Temp (F)_Weekly',
         'Shrubland_percentage Avg Air Temp (F)_month',
         'Shrubland_percentage Dew Point (F)_month',
         'Shrubland_percentage Max Air Temp (F)_month',
         'Shrubland_percentage Min Air Temp (F)_month',
         'Shrubland_percentage month__7',
         'Shrubland_percentage month__8',]

In [15]:
# Concatenate the X_train and y_train back into one training dataframe
training = pd.concat([X_train, y_train], axis=1)

In [16]:
# Split the target variable by class into two dataframes
no_fire = training[training.fire_started == 0] # 0 = No Wildfire
fire = training[training.fire_started == 1] # 1 = Wildfire

In [17]:
# Resample the minority class (wildfire)
fire_upsampled = resample(fire,
                          replace=True, # sample with replacement
                          n_samples=no_fire.shape[0], # match number in majority class
                          random_state=42) # reproducible result

In [18]:
# Create a new dataframe combining the target classes
resampled_df = pd.concat([no_fire, fire_upsampled])

In [19]:
# Double-check the different target classes
resampled_df.fire_started.value_counts()

1.0    12812
0.0    12812
Name: fire_started, dtype: int64

In [20]:
y_train = resampled_df.fire_started
X_train = resampled_df.drop(columns = ['fire_started'], axis = 1)

# Modeling

## Logistic Regression

### Base Model

In [21]:
# Instantiate a logistic regression model
logreg = LogisticRegression(random_state = 0) # random state for consistant results
# Train model on resampled training data
logreg.fit(X_train[selected_columns], y_train)

/Users/malcolmkatzenbach/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=0)

In [22]:
# Use the model to predict the target variable on the training dataset
y_hat_train = logreg.predict(X_train[selected_columns])
# Use the model to predict the target variable on the test dataset
y_hat_test = logreg.predict(X_test[selected_columns])

In [23]:
# Print the f1 score metric on both the training and test predictions to check for overfitting
print('F1 Score: Training', f1_score(y_train, y_hat_train), 'Test', f1_score(y_test, y_hat_test))
# Print the accuracy score metric on both the training and test predictions to check for overfitting
print('Accuracy Score: Training', accuracy_score(y_train, y_hat_train), 'Test', accuracy_score(y_test, y_hat_test))
# Print the recall score metric on both the training and test predictions to check for overfitting
print('Recall Score: Training', recall_score(y_train, y_hat_train), 'Test', recall_score(y_test, y_hat_test))

F1 Score: Training 0.6666666666666666 Test 0.12010768275005178
Accuracy Score: Training 0.5 Test 0.06389072482925755
Recall Score: Training 1.0 Test 1.0


In [24]:
confusion_matrix(y_test, y_hat_test)

array([[   0, 4249],
       [   0,  290]])

### Model - Scaled Data

In [25]:
# Insantiate the StandardScaler()
ss = StandardScaler()
# Fit the feature training data
ss.fit(X_train[selected_columns])

# Transform both the training and test features
X_train_scaled = ss.transform(X_train[selected_columns])
X_test_scaled = ss.transform(X_test[selected_columns])

In [26]:
# Instantiate a new logistic regression model
logreg1 = LogisticRegression(solver = 'liblinear')
# Fit the data to the new scaled data
logreg1.fit(X_train_scaled, y_train)

LogisticRegression(solver='liblinear')

In [27]:
# Use model to predict target variable on the training dataset
y_hat_train1 = logreg1.predict(X_train_scaled)
# Use model to predict target variable on the test dataset
y_hat_test1 = logreg1.predict(X_test_scaled)

In [28]:
# Print the f1 score metric on both the training and test predictions to check for overfitting
print('F1 Score: Training', f1_score(y_train, y_hat_train1), 'Test', f1_score(y_test, y_hat_test1))
# Print the accuracy score metric on both the training and test predictions to check for overfitting
print('Accuracy Score: Training', accuracy_score(y_train, y_hat_train1), 'Test', accuracy_score(y_test, y_hat_test1))
# Print the recall score metric on both the training and test predictions to check for overfitting
print('Recall Score: Training', recall_score(y_train, y_hat_train1), 'Test', recall_score(y_test, y_hat_test1))

F1 Score: Training 0.7963475787740438 Test 0.28031687995124926
Accuracy Score: Training 0.7876209803309397 Test 0.7398105309539547
Recall Score: Training 0.8304714330315329 Test 0.7931034482758621


In [29]:
confusion_matrix(y_test, y_hat_test1)

array([[3128, 1121],
       [  60,  230]])

In [30]:
scores = cross_val_score(logreg1, X_train_scaled, y_train, cv=10)
print(scores.mean())

0.7845376755564611


## K Nearest Neighbor (KNN)

### Base Model

In [31]:
knn = KNeighborsClassifier()

In [32]:
# Fit knn model using the scaled data from the previous scaled logistic model
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [33]:
# Predict target variable for both the train and test datasets.
knn_train = knn.predict(X_train_scaled)
knn_test = knn.predict(X_test_scaled)

In [34]:
# Print the f1 score metric on both the training and test predictions to check for overfitting
print('F1 Score: Training', f1_score(y_train, knn_train), 'Test', f1_score(y_test, knn_test))
# Print the sccuracy score metric on both the training and test predictions to check for overfitting
print('Accuracy Score: Training', accuracy_score(y_train, knn_train), 'Test', accuracy_score(y_test, knn_test))
# Print the recall score metric on both the training and test predictions to check for overfitting
print('Recall Score: Training', recall_score(y_train, knn_train), 'Test', recall_score(y_test, knn_test))

F1 Score: Training 0.9567262815965352 Test 0.2707070707070707
Accuracy Score: Training 0.9547689665938183 Test 0.8409341264595726
Recall Score: Training 1.0 Test 0.46206896551724136


In [35]:
confusion_matrix(y_test, knn_test)

array([[3683,  566],
       [ 156,  134]])

### Tuning Number of Nearest Neighbors

In [36]:
# Create a function to find the max recall score and return the score along with the k value
def max_value(l):
    max_val = max(l)
    max_idx = l.index(max_val)
    return max_idx, max_val

In [37]:
# Create an empty list for recall scores
k_scores = []
# Choose a range of k values to test
k_range = list(range(1, 21))
# Iterate through the different k values
for k in k_range:
    # Instantiate new knn model with k nearest neighbors
    knn = KNeighborsClassifier(n_neighbors = k)
    # Fit knn model on scaled training data
    knn.fit(X_train_scaled, y_train)
    # Use model to predict target variable on testing set
    y_pred = knn.predict(X_test_scaled)
    # Find the recall score
    recall = recall_score(y_test, y_pred)
    # Append recall score to list of recall scorees
    k_scores.append(recall)

# Find max recall score
idx, val = max_value(k_scores)
# Print max recall score and it corresponding k value
print(idx + 1, val)

19 0.7172413793103448


In [38]:
knn = KNeighborsClassifier(n_neighbors = 19)
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)
knn_train1 = knn.predict(X_train_scaled)
print('F1 Score: Training', f1_score(y_train, knn_train1), 'Test', f1_score(y_test, y_pred))
print('Accuracy Score: Training', accuracy_score(y_train, knn_train1), 'Test', accuracy_score(y_test, y_pred))
print('Recall Score: Training', recall_score(y_train, knn_train1), 'Test', recall_score(y_test, y_pred))

F1 Score: Training 0.8831105879094245 Test 0.23880597014925375
Accuracy Score: Training 0.8682485170152981 Test 0.7078651685393258
Recall Score: Training 0.9953949422416485 Test 0.7172413793103448


In [39]:
confusion_matrix(y_test, y_pred)

array([[3005, 1244],
       [  82,  208]])

In [40]:
scores_knn = cross_val_score(knn, X_train_scaled, y_train, cv=10)
print(scores_knn.mean())

0.845614039095359


## Decision Tree

### Base Model

In [41]:
dt = DecisionTreeClassifier(random_state = 0)
dt.fit(X_train[selected_columns], y_train)

DecisionTreeClassifier(random_state=0)

In [42]:
dt_train = dt.predict(X_train[selected_columns])
dt_test = dt.predict(X_test[selected_columns])

print('F1 Score: Training', f1_score(y_train, dt_train), 'Test', f1_score(y_test, dt_test))
print('Accuracy Score: Training', accuracy_score(y_train, dt_train), 'Test', accuracy_score(y_test, dt_test))
print('Recall Score: Training', recall_score(y_train, dt_train), 'Test', recall_score(y_test, dt_test))

F1 Score: Training 1.0 Test 0.2087912087912088
Accuracy Score: Training 1.0 Test 0.9048248512888302
Recall Score: Training 1.0 Test 0.19655172413793104


In [43]:
confusion_matrix(y_test, dt_test)

array([[4050,  199],
       [ 233,   57]])

### Training Parameters - GridSearchCV

In [44]:
parameters = {'max_depth' : range(10, 15, 1), 'max_features' : range(55, 75, 1), 'min_samples_split' : range(10, 20, 1)}

In [45]:
dtg = DecisionTreeClassifier(random_state = 0)
grid_model = GridSearchCV(dtg, parameters, cv = 10, scoring = 'f1', verbose = 1, n_jobs = -1)

grid_model.fit(X_train[selected_columns], y_train)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:   50.1s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 4968 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 6018 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 7168 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 8418 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 9768 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 10000 out of 1000

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': range(10, 15),
                         'max_features': range(55, 75),
                         'min_samples_split': range(10, 20)},
             scoring='f1', verbose=1)

In [46]:
print(grid_model.best_score_)
print(grid_model.best_params_)
print(grid_model.best_estimator_)

0.9238253819497579
{'max_depth': 14, 'max_features': 57, 'min_samples_split': 13}
DecisionTreeClassifier(max_depth=14, max_features=57, min_samples_split=13,
                       random_state=0)


In [47]:
dt_train2 = grid_model.best_estimator_.predict(X_train[selected_columns])
dt_test2 = grid_model.best_estimator_.predict(X_test[selected_columns])

print('F1 Score: Training', f1_score(y_train, dt_train2), 'Test', f1_score(y_test, dt_test2))
print('Accuracy Score: Training', accuracy_score(y_train, dt_train2), 'Test', accuracy_score(y_test, dt_test2))
print('Recall Score: Training', recall_score(y_train, dt_train2), 'Test', recall_score(y_test, dt_test2))

F1 Score: Training 0.9446270917217686 Test 0.26149684400360684
Accuracy Score: Training 0.9415001561036528 Test 0.8195637805684072
Recall Score: Training 0.9979706525132688 Test 0.5


In [63]:
parameters2 = {'max_depth' : range(5, 6, 1), 'max_features' : range(55, 75, 1), 'min_samples_split' : range(10, 20, 1)}

In [64]:
dtg2 = DecisionTreeClassifier(random_state = 0)
dtg_model2 = GridSearchCV(dtg2, parameters2, cv = 10, scoring = 'f1', verbose = 1, n_jobs = -1)

dtg_model2.fit(X_train[selected_columns], y_train)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:   47.9s
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  1.4min finished


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': range(5, 6),
                         'max_features': range(55, 75),
                         'min_samples_split': range(10, 20)},
             scoring='f1', verbose=1)

In [65]:
print(dtg_model2.best_score_)
print(dtg_model2.best_params_)
print(dtg_model2.best_estimator_)

0.8156761079336213
{'max_depth': 5, 'max_features': 70, 'min_samples_split': 10}
DecisionTreeClassifier(max_depth=5, max_features=70, min_samples_split=10,
                       random_state=0)


In [66]:
dt_train3 = dtg_model2.best_estimator_.predict(X_train[selected_columns])
dt_test3 = dtg_model2.best_estimator_.predict(X_test[selected_columns])

print('F1 Score: Training', f1_score(y_train, dt_train3), 'Test', f1_score(y_test, dt_test3))
print('Accuracy Score: Training', accuracy_score(y_train, dt_train3), 'Test', accuracy_score(y_test, dt_test3))
print('Recall Score: Training', recall_score(y_train, dt_train3), 'Test', recall_score(y_test, dt_test3))

F1 Score: Training 0.8210962489095667 Test 0.2672209026128266
Accuracy Score: Training 0.8079144551982517 Test 0.7281339502092972
Recall Score: Training 0.8815953793318764 Test 0.7758620689655172


In [67]:
confusion_matrix(y_test, dt_test3)

array([[3080, 1169],
       [  65,  225]])

## Random Forest Classifier

### Base Model

In [69]:
rfc = RandomForestClassifier(random_state = 0)
rfc.fit(X_train[selected_columns], y_train)

RandomForestClassifier(random_state=0)

In [70]:
rfc_train = rfc.predict(X_train[selected_columns])
rfc_test = rfc.predict(X_test[selected_columns])

In [71]:
print('F1 Score: Training', f1_score(y_train, rfc_train), f1_score(y_test, rfc_test))
print('Accuracy Score: Training', accuracy_score(y_train, rfc_train), accuracy_score(y_test, rfc_test))
print('Recall Score: Training', recall_score(y_train, rfc_train), recall_score(y_test, rfc_test))

F1 Score: Training 1.0 0.2517162471395881
Accuracy Score: Training 1.0 0.9279576999339062
Recall Score: Training 1.0 0.1896551724137931


In [72]:
confusion_matrix(y_test, rfc_test)

array([[4157,   92],
       [ 235,   55]])

### Tuning Parameters - GridSearchCV

In [73]:
param_grid = { 
    'n_estimators': [100,300],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': list(range(2,10)),
    'max_features': list(range(3,7))
}

In [74]:
grid_tree=GridSearchCV(RandomForestClassifier(random_state = 0), param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)

In [75]:
grid_tree.fit(X_train[selected_columns], y_train)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 640 out of 640 | elapsed: 11.0min finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': [3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9],
                         'n_estimators': [100, 300]},
             scoring='f1', verbose=1)

In [76]:
print(grid_tree.best_score_)
print(grid_tree.best_params_)
print(grid_tree.best_estimator_)

0.9894985877845526
{'criterion': 'gini', 'max_features': 3, 'min_samples_split': 4, 'n_estimators': 300}
RandomForestClassifier(max_features=3, min_samples_split=4, n_estimators=300,
                       random_state=0)


In [78]:
grid_tree_train = grid_tree.best_estimator_.predict(X_train[selected_columns])
grid_tree_test = grid_tree.best_estimator_.predict(X_test[selected_columns])
print('F1 Score: Training', f1_score(y_train, grid_tree_train), f1_score(y_test, grid_tree_test))
print('Accuracy Score: Training', accuracy_score(y_train, grid_tree_train), accuracy_score(y_test, grid_tree_test))
print('Recall Score: Training', recall_score(y_train, grid_tree_train), recall_score(y_test, grid_tree_test))

F1 Score: Training 1.0 0.22660098522167493
Accuracy Score: Training 1.0 0.9308217669090108
Recall Score: Training 1.0 0.15862068965517243


In [79]:
confusion_matrix(y_test, grid_tree_test)

array([[4179,   70],
       [ 244,   46]])

In [80]:
param_grid2 = { 
    'n_estimators': [75, 100, 125],
    'criterion': ['gini'],
    'min_samples_split': list(range(1200, 1202)),
    'max_features': list(range(30, 40))
}

In [81]:
grid_tree2=GridSearchCV(RandomForestClassifier(random_state = 0), param_grid2, cv=5, scoring='f1', verbose=1, n_jobs=-1)

In [82]:
grid_tree2.fit(X_train[selected_columns], y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   36.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  9.2min finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'criterion': ['gini'],
                         'max_features': [30, 31, 32, 33, 34, 35, 36, 37, 38,
                                          39],
                         'min_samples_split': [1200, 1201],
                         'n_estimators': [75, 100, 125]},
             scoring='f1', verbose=1)

In [83]:
print(grid_tree2.best_score_)
print(grid_tree2.best_params_)
print(grid_tree2.best_estimator_)

0.807252252823665
{'criterion': 'gini', 'max_features': 39, 'min_samples_split': 1201, 'n_estimators': 75}
RandomForestClassifier(max_features=39, min_samples_split=1201, n_estimators=75,
                       random_state=0)


In [84]:
grid_tree_train2 = grid_tree2.best_estimator_.predict(X_train[selected_columns])
grid_tree_test2 = grid_tree2.best_estimator_.predict(X_test[selected_columns])
print('F1 Score: Training', f1_score(y_train, grid_tree_train2), f1_score(y_test, grid_tree_test2))
print('Accuracy Score: Training', accuracy_score(y_train, grid_tree_train2), accuracy_score(y_test, grid_tree_test2))
print('Recall Score: Training', recall_score(y_train, grid_tree_train2), recall_score(y_test, grid_tree_test2))

F1 Score: Training 0.8183619047619046 0.2903453136011276
Accuracy Score: Training 0.8139244458320325 0.7781449658515092
Recall Score: Training 0.8383546674992195 0.7103448275862069


In [86]:
confusion_matrix(y_test, grid_tree_test2)

array([[3326,  923],
       [  84,  206]])

In [101]:
param_grid3 = { 
    'n_estimators': [75, 100, 125],
    'criterion': ['gini'],
    'min_samples_split': list(range(2000, 2002)),
    'max_features': list(range(40, 50))
}

In [102]:
grid_tree3=GridSearchCV(RandomForestClassifier(random_state = 0), param_grid3, cv=5, scoring='f1', verbose=1, n_jobs=-1)

In [103]:
grid_tree3.fit(X_train[selected_columns], y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   36.5s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  9.0min finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'criterion': ['gini'],
                         'max_features': [40, 41, 42, 43, 44, 45, 46, 47, 48,
                                          49],
                         'min_samples_split': [2000, 2001],
                         'n_estimators': [75, 100, 125]},
             scoring='f1', verbose=1)

In [104]:
print(grid_tree3.best_score_)
print(grid_tree3.best_params_)
print(grid_tree3.best_estimator_)

0.7774430629885181
{'criterion': 'gini', 'max_features': 40, 'min_samples_split': 2000, 'n_estimators': 100}
RandomForestClassifier(max_features=40, min_samples_split=2000, random_state=0)


In [105]:
grid_tree_train3 = grid_tree3.best_estimator_.predict(X_train[selected_columns])
grid_tree_test3 = grid_tree3.best_estimator_.predict(X_test[selected_columns])
print('F1 Score: Training', f1_score(y_train, grid_tree_train3), f1_score(y_test, grid_tree_test3))
print('Accuracy Score: Training', accuracy_score(y_train, grid_tree_train3), accuracy_score(y_test, grid_tree_test3))
print('Recall Score: Training', recall_score(y_train, grid_tree_train3), recall_score(y_test, grid_tree_test3))

F1 Score: Training 0.7892284186401833 0.27676240208877284
Accuracy Score: Training 0.7846550109272556 0.7558933685833884
Recall Score: Training 0.8063534186699969 0.7310344827586207
